# Preparing data for yolo training




In [1]:
import torch
from IPython.display import Image  # for displaying images
import os 
import numpy as np
import pandas as pd
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET #not used
from xml.dom import minidom #not used
from tqdm import tqdm
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import pathlib

from animal_helpers import *

import cv2 #open cv for plotting the images with bounding boxes
random.seed(108)

%matplotlib inline

image_folder = "./kga_images/KGA_public"
all_files = list_files_recursively(image_folder )[0:10]
#for file in all_files:
#    print(file)

# Prepare files for training

The images are in a file structure so need to be reorganised into a train test and validation set.

- A folder structure will be created
- A meta data file on only images with animals will be created
- The metadata file will be sampled and split into train/test/valid
- The meta data file will have the original and new paths added
- The files will be copied to the new locations

Once the set up is complete the following process will be performed

- Run Megadetector on the train test and validation folder
- Create a bounding box text tiles for each of the image files


In [2]:
# Create the new folders

for y in ['images/', 'labels/']:
    for folder in list(map(lambda x: y + x, ['train', 'test'])):
        full_path = os.path.join('./kga_images',folder)
        if not os.path.exists(full_path):
            os.makedirs(full_path)

In [3]:

meta_image_inv = pd.read_csv('./kga_images/KGA_S1_report_lila_image_inventory.csv')
meta_data = pd.read_csv('./kga_images/KGA_S1_report_lila.csv')
meta_data = meta_data[meta_data['question__species']!= 'blank']
meta_data = meta_data[meta_data['question__species']!= 'human']
class_map = pd.read_csv('./kga_images/kga_class_map.csv')


meta_image_inv = meta_image_inv.merge(
    meta_data.loc[:,['capture_id', 'question__species', 'site']], 
    how = 'left',
    on = 'capture_id').merge(
    class_map,
    how = 'left',
    on = 'question__species'
)
meta_image_inv = meta_image_inv[meta_image_inv['question__species'].notnull()]

print(meta_data.shape)
print(meta_image_inv.shape)

(909, 23)
(2381, 8)


In [4]:

#total number of species
meta_data.groupby(['question__species']).size()

question__species
aardvarkantbear         1
birdofprey              4
birdother             108
bustardkori            34
caracal                 4
catafricanwild         10
cheetah                 1
duikercommongrey        9
eland                  14
foxbateared             1
foxcape                19
gemsbokoryx           502
harecape                1
hartebeestred           5
honeybadger             3
hyenabrown             11
hyenaspotted            1
jackalblackbacked      39
kudu                    6
leopard                 3
lionfemale              1
meerkatsuricate         1
ostrich                46
porcupine               3
reptilesamphibians      1
rodent                  1
secretarybird           9
steenbok               58
wildebeestblue         13
dtype: int64

In [5]:
#table of species vs cameras

#black backed jackel and possibly cape fox are good target species for boosting model performance
meta_image_inv.groupby(['type', 'site']).size().reset_index().pivot_table(values=0, index='type', columns='site')#.sort_values('counts')

site,A01,A02,A03,A04,A05,A06,A07,A08,A09,A10,A11,B01,B02,B03,B04,B05,B06,B07,B08,B09
type,,,,,,,,,,,,,,,,,,,,
bird,141.0,NaN,1.0,27.0,12.0,9.0,12.0,21.0,NaN,3.0,6.0,12.0,NaN,NaN,6.0,NaN,33.0,99.0,3.0,78.0
cat_thing,NaN,NaN,NaN,NaN,3.0,NaN,1.0,1.0,NaN,NaN,7.0,8.0,3.0,NaN,NaN,3.0,1.0,NaN,1.0,1.0
cow_thing,86.0,69.0,9.0,117.0,79.0,299.0,27.0,79.0,124.0,8.0,27.0,28.0,71.0,11.0,10.0,14.0,NaN,11.0,NaN,550.0
dog_thing,6.0,NaN,NaN,3.0,9.0,3.0,14.0,4.0,NaN,8.0,9.0,15.0,15.0,9.0,1.0,NaN,2.0,13.0,4.0,NaN
ostrich,114.0,NaN,NaN,9.0,NaN,NaN,NaN,6.0,1.0,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
other,5.0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,2.0,1.0,1.0,1.0,NaN,NaN,3.0,NaN,NaN,NaN


In [6]:
#Megadetector is a 3 class YOLOv5x6 model. aka the yoloVt6 produced by the YOLOv5 team
#Model is here and can be downloading using wget in the terminal
#https://github.com/microsoft/CameraTraps/releases/download/v5.0/md_v5a.0.0.pt
import sys
from contextlib import contextmanager
@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

# Suppressing the verbose output
with suppress_stdout():
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='md_v5a.0.0.pt')


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-1-14 Python-3.9.13 torch-1.12.0+cu116 CUDA:0 (Quadro P5000, 16273MiB)

Fusing layers... 
Model summary: 574 layers, 139990096 parameters, 0 gradients
Adding AutoShape... 


In [7]:
meta_image_inv

,capture_id,image_rank_in_capture,image_path_rel,question__species,site,counts,type,class
15,KGA_S1#A01#1#8,1,KGA_S1/A01/A01_R1/KGA_S1_A01_R1_IMAG0018.JPG,gemsbokoryx,A01,502.0,cow_thing,2.0
16,KGA_S1#A01#1#9,1,KGA_S1/A01/A01_R1/KGA_S1_A01_R1_IMAG0019.JPG,gemsbokoryx,A01,502.0,cow_thing,2.0
17,KGA_S1#A01#1#10,1,KGA_S1/A01/A01_R1/KGA_S1_A01_R1_IMAG0020.JPG,gemsbokoryx,A01,502.0,cow_thing,2.0
18,KGA_S1#A01#1#11,1,KGA_S1/A01/A01_R1/KGA_S1_A01_R1_IMAG0021.JPG,gemsbokoryx,A01,502.0,cow_thing,2.0
19,KGA_S1#A01#1#12,1,KGA_S1/A01/A01_R1/KGA_S1_A01_R1_IMAG0022.JPG,gemsbokoryx,A01,502.0,cow_thing,2.0
...,...,...,...,...,...,...,...,...
10394,KGA_S1#B09#1#213,3,KGA_S1/B09/B09_R1/KGA_S1_B09_R1_IMAG0631.JPG,steenbok,B09,58.0,cow_thing,2.0
10395,KGA_S1#B09#1#214,1,KGA_S1/B09/B09_R1/KGA_S1_B09_R1_IMAG0632.JPG,steenbok,B09,58.0,cow_thing,2.0
10396,KGA_S1#B09#1#214,2,KGA_S1/B09/B09_R1/KGA_S1_B09_R1_IMAG0633.JPG,steenbok,B09,58.0,cow_thing,2.0
10397,KGA_S1#B09#1#214,3,KGA_S1/B09/B09_R1/KGA_S1_B09_R1_IMAG0634.JPG,steenbok,B09,58.0,cow_thing,2.0


## Running yolo to find bounding boxes

THe below chunk runs megadetector over the images to identify the bounding boxes. As the images have allready been classified we know the class of what ever is found. We therefore create a new dataframe that contains the bounds and class. This will allow us to create a training and test dataset. The code is relatively slow, so the code only runs if the base file is not there

In [8]:
batch_size = 100  # Adjust based on your GPU's capacity and image sizes
batches = create_batches(meta_image_inv, batch_size)
results_list = [process_batch(batch, image_folder, model) for batch in batches]
#results_list = [result for result in results_list if result is not None]

output_df = pd.concat(results_list, ignore_index=True) if results_list else pd.DataFrame()

print('bounding boxes, found detection complete')

batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
batch complete returning batch
bounding boxes, found detection complete


In [9]:

files_with_boxes = pd.read_csv('image_boundingboxes.csv')

#file_list  = meta_image_inv.image_path_rel[meta_image_inv.capture_id.str.contains("B07|A08")].values
file_list  = files_with_boxes.file_name[files_with_boxes.file_name.str.contains("B07|A08")].values

#absolute paths of images and txt files
image_files_list = [os.path.join(image_folder, x) for x in file_list]
txt_file_list = [os.path.join('./kga_images/kga_txt', x).replace('.JPG', '.txt') for x in file_list]

#populate_with_copies(image_files_list,'./kga_images/images/test/')
populate_with_symlinks(txt_file_list ,'./kga_images/labels/test/')

print('test data complete')

Total symbolic links created: 187
test data complete


In [10]:

files_with_boxes = pd.read_csv('image_boundingboxes.csv')

#file_list  = meta_image_inv.image_path_rel[meta_image_inv.capture_id.str.contains("B07|A08")].values
file_list  = files_with_boxes.file_name[files_with_boxes.file_name.str.contains("B07|A08")==False].values

#absolute paths of images and txt files
image_files_list = [os.path.join(image_folder, x) for x in file_list]
txt_file_list = [os.path.join('./kga_images/kga_txt', x).replace('.JPG', '.txt') for x in file_list]

#populate_with_copies(image_files_list,'./kga_images/images/train/')
populate_with_symlinks(txt_file_list ,'./kga_images/labels/train/')

print('test data complete')

Total symbolic links created: 2396
test data complete


# Training the Yolo model

Open a tensorboard with the following

- run `tensorboard --logdir . --bind_all` in the terminal
- navigate to the tensorboad location. in paperspace this will be somethiing similar too https://tensorboard-XXXX.clg07azjl.paperspacegradient.com where XXX is the cluster id
- then from the root directory in the terminal run `python ./yolov5/train.py --data ./yolov5/data/kga.yaml --weights md_v5a.0.0.pt`
- If for some reason training terminates earlier than you want. you can run `python ./yolov5/train.py --data ./yolov5/data/kga.yaml --weights ./yolov5/runs/train/expX/weights/last.pt --resume` where `expX` is the model id


# Import additional Inaturalist files from GBIF

This requires the installation of the gbif_dl from plantnet and the copying of the doi given when I specified the datafile

The idea with this section is to train a model that can detect lions even though there is almost no lion data

The concept is essentially transfer learning.The key is that because we are not trying to predict the whole image but instead only the bounding box and because yolo combines multiple images into a single training image, we can avoid the issues associated with predicting the background. A bit of a moon shot but could work.

In [11]:
import pandas as pd
test  = pd.read_csv('occurrence.txt', sep = '\s+')

In [12]:
test.filter(regex = 'species').columns

Index(['speciesKey', 'species'], dtype='object')

In [13]:
test

,gbifID,accessRights,bibliographicCitation,language,license,...,level2Name,level3Gid,level3Name,iucnRedListCategory,eventType
0,4506557620,CC_BY_NC_4_0,2023-12-27T13:22:57Z,https://www.inaturalist.org/observations/19443...,Mark,...,NaN,NaN,NaN,NaN,NaN
1,4506553015,CC_BY_NC_4_0,2023-12-27T13:14:01Z,https://www.inaturalist.org/observations/19488...,Pierre-Louis,...,NaN,NaN,NaN,NaN,NaN
2,4506547710,CC_BY_NC_4_0,2023-12-26T05:09:35Z,https://www.inaturalist.org/observations/19480...,Pierre-Louis,...,NaN,NaN,NaN,NaN,NaN
3,4506547441,CC_BY_NC_4_0,2023-12-27T13:17:46Z,https://www.inaturalist.org/observations/19471...,jakeh2022,...,NaN,NaN,NaN,NaN,NaN
4,4506546010,CC_BY_NC_4_0,2023-12-27T21:07:40Z,https://www.inaturalist.org/observations/19488...,Pierre-Louis,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2677,2557804868,CC_BY_4_0,2023-03-31T21:08:01Z,https://www.inaturalist.org/observations/37615326,Christiaan,...,NaN,NaN,NaN,NaN,NaN
2678,2557781885,CC_BY_NC_4_0,2023-03-31T21:39:34Z,https://www.inaturalist.org/observations/37550725,Mwangi,...,NaN,NaN,NaN,NaN,NaN
2679,2557770955,CC_BY_4_0,2023-03-31T21:47:50Z,https://www.inaturalist.org/observations/37521579,Christiaan,...,NaN,NaN,NaN,NaN,NaN
2680,2557769319,CC_BY_4_0,2023-03-31T21:48:33Z,https://www.inaturalist.org/observations/37517027,marius,...,NaN,NaN,NaN,NaN,NaN


In [14]:
test.groupby(['speciesKey']).size()

Series([], dtype: int64)